In [6]:
# Cell 1: Setup with your existing structure

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
from PIL import Image
from tqdm import tqdm
import pandas as pd

sys.path.append('..')
from src.data import create_dataloaders, denormalize

# Attribution imports
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import torchvision.models as models
import timm
from captum.attr import Saliency, IntegratedGradients, Occlusion

# Load config
# Load config
with open('../experiments/configs/baseline_config.json', 'r') as f:
    full_config = json.load(f)

CONFIG = full_config['resnet50']  # or 'deit_small' / 'swin_tiny'

device = torch.device(CONFIG.get('device', 'cuda' if torch.cuda.is_available() else 'cpu'))

train_loader, val_loader, dataset = create_dataloaders(
    data_root=CONFIG['data']['data_root'],
    split_metadata_path=CONFIG['data']['split_metadata_path'],
    batch_size=CONFIG['training']['batch_size'],
    num_workers=CONFIG['training']['num_workers'],
    img_size=CONFIG['data']['image_size'],
    pin_memory=False
)
val_dataset = val_loader.dataset

print(f"✓ Validation dataset: {len(val_dataset)} samples")
print(f"✓ Device: {device}")
print(f"✓ Num classes: {CONFIG['num_classes']}")

# Create output directories
output_base = '../experiments/results/attribution_analysis'
os.makedirs(output_base, exist_ok=True)

print(f"✓ Output directory: {output_base}")


RuntimeError: Dataset not found. You can use download=True to download it